In [1]:
import pandas as pd 
import numpy as np 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arushdwivedi/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
tweet_data= pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
print(tweet_data.shape)
tweet_data.head()


(1599999, 6)


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
column_names=['target','id','date','flag','user','text']
tweet_data= pd.read_csv('training.1600000.processed.noemoticon.csv', names=column_names ,encoding='ISO-8859-1')


In [5]:
tweet_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [6]:
tweet_data.target.value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert 4 target value to 1

In [7]:
tweet_data.replace({'target':{4:1}}, inplace=True)

0->negative
1->positive

In [8]:
tweet_data.target.value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

stemming:

In [9]:
port_stem=PorterStemmer()
def stemming(content):
    stemmed_content= re.sub('[^a-zA-Z]', ' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)

    return stemmed_content

tweet_data['stemmed content']=tweet_data['text'].apply(stemming)
    

In [10]:
tweet_data.head()

,target,id,date,flag,user,text,stemmed content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [11]:
#seperatin data and target
x=tweet_data['stemmed content'].values
y=tweet_data['target'].values 


In [12]:
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
print(xtrain.shape, xtest.shape)

(1280000,) (320000,)


In [13]:
vectorizer= TfidfVectorizer()
xtrain=vectorizer.fit_transform(xtrain)
xtest=vectorizer.transform(xtest)

In [14]:
logistic=LogisticRegression(max_iter=1000)
logistic.fit(xtrain,ytrain)
trainpred=logistic.predict(xtrain)
testpred=logistic.predict(xtest)


In [15]:

xtrainacc=accuracy_score(ytrain,trainpred)
xtestacc=accuracy_score(ytest,testpred)

In [16]:
print('training accuracy:', xtrainacc)
print('testing accuracy:', xtestacc)

training accuracy: 0.79871953125
testing accuracy: 0.77668125


saving the model

In [17]:
import pickle
filename='trainedmodel.sav'
pickle.dump(logistic,open(filename,'wb'))

LOADING THE MODEL

In [18]:
loadedmodel=pickle.load(open('trainedmodel.sav','rb'))

In [19]:
sampletweet = ["GOOD MORNING EVERYONE! I LOVE YOU ALL!!!"]
sampletweet = [stemming(tweet) for tweet in sampletweet] 

In [20]:
sampletweet=vectorizer.transform(sampletweet)

In [23]:
sampletweetpred= loadedmodel.predict(sampletweet)
if sampletweetpred==1:
    print('positive tweet')
else:
    print('negative tweet')

positive tweet


In [24]:
filename='vectorizer.sav'
pickle.dump(vectorizer,open(filename,'wb'))

In [26]:
vectormodel=pickle.load(open('vectorizer.sav','rb'))

In [27]:
testingtweet=["MY LIFE IS SO SAD OH MY GOD!"]
testingtweet=[stemming(tweet) for tweet in testingtweet]
testingtweet=vectormodel.transform(testingtweet)
testingtweetpred=loadedmodel.predict(testingtweet)
if testingtweetpred==1:
    print('positive tweet')
else:
    print('negative tweet')

negative tweet
